In [1]:
import torch
import torch.nn.functional as F
from torch import nn

In [2]:
a = torch.tensor([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
print(a)
# https://stackoverflow.com/questions/71723788/how-to-reverse-order-of-rows-in-a-tensor
# https://pytorch.org/docs/stable/generated/torch.flip.html
b = torch.flip(a, [0])
print(b)

tensor([[1, 2, 3],
        [4, 5, 6],
        [7, 8, 9]])
tensor([[7, 8, 9],
        [4, 5, 6],
        [1, 2, 3]])


In [3]:
x = torch.tensor([1, 2, 0])
F.one_hot(x, 4)

tensor([[0, 1, 0, 0],
        [0, 0, 1, 0],
        [1, 0, 0, 0]])

In [4]:
x = torch.tensor([2])
F.one_hot(x, 3)

tensor([[0, 0, 1]])

In [5]:
# 2023/9/28
# zhangzhong

In [6]:
# https://pytorch.org/docs/stable/generated/torch.nn.functional.one_hot.html
x = torch.tensor([[1, 2], [0, 3]])
x_one_hot = F.one_hot(x, 4)

# 其实就是之前对应位置的某一个数字 变成一个一个one_hot vector
print(x[0, 0], x_one_hot[0, 0])
print(x[0, 1], x_one_hot[0, 1])
print(x[1, 0], x_one_hot[1, 0])
print(x[1, 1], x_one_hot[1, 1])

tensor(1) tensor([0, 1, 0, 0])
tensor(2) tensor([0, 0, 1, 0])
tensor(0) tensor([1, 0, 0, 0])
tensor(3) tensor([0, 0, 0, 1])


In [7]:
# 再尝试一下交换维度
# batch_size = 2, num_seq = 3
x = torch.tensor([[0, 1, 2], [3, 4, 5]])
x_one_hot = F.one_hot(x, 6)
print(x_one_hot)

# 现在我们想遍历所有的样本 
for x in x_one_hot:
    print(x)

tensor([[[1, 0, 0, 0, 0, 0],
         [0, 1, 0, 0, 0, 0],
         [0, 0, 1, 0, 0, 0]],

        [[0, 0, 0, 1, 0, 0],
         [0, 0, 0, 0, 1, 0],
         [0, 0, 0, 0, 0, 1]]])
tensor([[1, 0, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0],
        [0, 0, 1, 0, 0, 0]])
tensor([[0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 1, 0],
        [0, 0, 0, 0, 0, 1]])


In [8]:
# 完全符合我们的推导 太对了!!
# https://pytorch.org/docs/stable/generated/torch.transpose.html
# Returns a tensor that is a transposed version of input. The given dimensions dim0 and dim1 are swapped.
x_t = torch.transpose(x_one_hot, 0, 1)
for x in x_t:
    print(x)

tensor([[1, 0, 0, 0, 0, 0],
        [0, 0, 0, 1, 0, 0]])
tensor([[0, 1, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 0]])
tensor([[0, 0, 1, 0, 0, 0],
        [0, 0, 0, 0, 0, 1]])


In [9]:
# yield from
def funa():
    yield 1
    yield 2
    yield 3


def funb():
    yield from funa()
    yield 4
    yield 5


for x in funb():
    print(x)

1
2
3
4
5


In [10]:
# list of Tensor can not convert to a whole tensor
# outputs: list[Tensor] = []
# outputs.append(torch.tensor([1,2,3]))
# outputs.append(torch.tensor([1,2,3]))
# outputs.append(torch.tensor([1,2,3]))
# outputs.append(torch.tensor([1,2,3]))
# output = torch.tensor(outputs)

# so you can use torch.stack
# https://pytorch.org/docs/stable/generated/torch.stack.html
# Concatenates a sequence of tensors along a new dimension.
# All tensors need to be of the same size.

x = torch.tensor([1, 2, 3])
xs = [x, x, x, x]
torch.stack(xs)

tensor([[1, 2, 3],
        [1, 2, 3],
        [1, 2, 3],
        [1, 2, 3]])

In [11]:
x = torch.tensor([[1, 2, ], [3, 4, ]])
xs = [x, x, x, x]
torch.stack(xs)

tensor([[[1, 2],
         [3, 4]],

        [[1, 2],
         [3, 4]],

        [[1, 2],
         [3, 4]],

        [[1, 2],
         [3, 4]]])

In [12]:
y = torch.stack(xs, dim=1)
y.shape, y

(torch.Size([2, 4, 2]),
 tensor([[[1, 2],
          [1, 2],
          [1, 2],
          [1, 2]],
 
         [[3, 4],
          [3, 4],
          [3, 4],
          [3, 4]]]))

In [13]:
# 向量各分量的平方和 就是 l2范数
# x = torch.tensor([1,2,3])
# x.norm(1)

In [14]:
# grad clip
# 二维矩阵的grad长什么样子
x = torch.tensor([[1, 2], [3, 4]], dtype=torch.float32, requires_grad=True)
b = torch.tensor([1, 2], dtype=torch.float32, requires_grad=True)
y = x + b
y.backward(torch.ones_like(x))
print(x.grad)


tensor([[1., 1.],
        [1., 1.]])


In [15]:
# test sum along some axis
x1 = torch.tensor([[1, 2, 3], [4, 5, 6]])
x2 = torch.tensor([[4, 5, 6], [1, 2, 3]])
x3 = torch.tensor([[3, 2, 3], [1, 3, 4]])
x = torch.stack([x1, x2, x3])
print(x)

# now we need to add the two 2x3 matrix, 好像是对的
torch.sum(input=x, dim=0)

tensor([[[1, 2, 3],
         [4, 5, 6]],

        [[4, 5, 6],
         [1, 2, 3]],

        [[3, 2, 3],
         [1, 3, 4]]])


tensor([[ 8,  9, 12],
        [ 6, 10, 13]])

In [16]:
# AdaptiveAvgPool2d
# https://pytorch.org/docs/stable/generated/torch.nn.AdaptiveAvgPool2d.html
pool = nn.AdaptiveAvgPool2d(output_size=(5, 7))
input = torch.randn(size=(1, 64, 8, 9))
output = pool(input)
print(output.shape)
# 哦 原来如此 channel是不会变的 指定输出的最后两个维度

# 实际上是每个panel上的所有元素做一下avgrage 这样最终输出的就是一个num class 的 logits 然后过cross entropy就可以分类了
# 自己构造数据看一下实际的效果 (1, 3, 2, 2)
input = torch.tensor(data=[
    [[[1, 2], [3, 4]], [[2, 3], [1, 4]], [[3, 4], [3, 2]]]
], dtype=torch.float)
assert input.shape == (1, 3, 2, 2)
pool = nn.AdaptiveAvgPool2d(output_size=(1, 1))
output = pool(input)
assert output.shape == (1, 3, 1, 1)
print(output)
flatten = nn.Flatten()
output = flatten(output)
assert output.shape == (1, 3)
assert torch.all(output == torch.tensor([[2.5000, 2.5000, 3.0000]]))
print(output)

torch.Size([1, 64, 5, 7])
tensor([[[[2.5000]],

         [[2.5000]],

         [[3.0000]]]])
tensor([[2.5000, 2.5000, 3.0000]])


In [17]:
# conv2d batch norm
x = torch.randn(size=(4, 3, 4, 4))
y = x.mean(dim=1)
print(y.shape)

# 好像就剩下channel了
y1 = x.mean(dim=(0, 2, 3))
print(y1.shape)

y2 = x.mean(dim=(0, 2, 3), keepdim=True)
print(y2.shape)

torch.Size([4, 4, 4])
torch.Size([3])
torch.Size([1, 3, 1, 1])


In [18]:
# split and cat
x = torch.tensor(data=[[1, 2], [3, 4]])
y = x.split(split_size=1, dim=0)
print(y)
z = torch.cat(y, dim=0)
print(z)
# t的类型推断是错误的 应该是tensor 但是推断成了bool
t = (z == x)
print(t)

y = x.split(split_size=1, dim=1)
print(y)
z = torch.cat(y, dim=1)
print(z)
assert torch.all(z == x)


(tensor([[1, 2]]), tensor([[3, 4]]))
tensor([[1, 2],
        [3, 4]])
tensor([[True, True],
        [True, True]])
(tensor([[1],
        [3]]), tensor([[2],
        [4]]))
tensor([[1, 2],
        [3, 4]])


In [19]:
torch.range(start=0, end=12)

/tmp/ipykernel_9788/1580358168.py:1: UserWarning: torch.range is deprecated and will be removed in a future release because its behavior is inconsistent with Python's range builtin. Instead, use torch.arange, which produces values in [start, end).
  torch.range(start=0, end=12)


tensor([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12.])

In [21]:
# check the output of conv
batch_size = 32
out_channels = 64

conv = nn.LazyConv2d(out_channels=out_channels, kernel_size=7, stride=2, padding=3)
x = torch.randn(size=(batch_size, 3, 32, 32))
y = conv(x)
print(y.shape)

pool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
y = pool(y)
print(y.shape)

torch.Size([32, 64, 16, 16])
torch.Size([32, 64, 8, 8])


/data/zhangzhong/anaconda3/envs/ml/lib/python3.10/site-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


In [33]:
import operator, itertools
import functools

sum = 0
for p in conv.parameters():
    # https://pytorch.org/docs/stable/generated/torch.nn.parameter.Parameter.html
    # p.data: Tensor
    print(p.shape, p.data.dtype)
    # https://docs.python.org/3.0/library/functools.html#functools.reduce
    sum += functools.reduce(operator.mul, p.shape)
    
# 算了 统计类型其实没啥意义 就单纯看看参数量就行了
print(sum)


torch.Size([64, 3, 7, 7]) torch.float32
torch.Size([64]) torch.float32
9472


In [42]:
from module.vision.resnet import ResNet18

net = ResNet18()
net.summary(shape=(1, 3, 96, 96))

input shape:  (1, 3, 96, 96)
Sequential output shape:  torch.Size([1, 64, 24, 24]) , parameters:  9600
ResNetBlock output shape:  torch.Size([1, 64, 24, 24]) , parameters:  74112


RuntimeError: The size of tensor a (128) must match the size of tensor b (64) at non-singleton dimension 1